In [135]:
from collections import Counter
import pandas as pd

In [2]:
with open('./DATASET/train/truthful.txt') as t, open('./DATASET/train/deceptive.txt') as d:
    truthful = t.read()
    deceptive = d.read()

In [71]:
def clean_text(words):
    words = '<s> ' + words
    words = words.replace('\n', ' <s> ')
    return words[:-5]

## Make Unigrams

In [73]:
truthful_unigram = dict(Counter(clean_text(truthful).split()))
deceptive_unigram = dict(Counter(clean_text(deceptive).split()))

In [74]:
def get_unigram_prob(corpus, unigram_to_test):
    total_words = 0
    for key in corpus:
        total_words += corpus[key]

    return corpus[unigram_to_test]/total_words

In [75]:
get_unigram_prob(deceptive_unigram, 'I')

0.02905073649754501

## Make Bigrams

In [80]:
def get_bigram_counts(word_list):
    corpus = {}
    for i, word in enumerate(word_list[1:], start=1):
        if word != '<s>':
            if (word_list[i-1], word) not in corpus:
                corpus[(word_list[i-1], word)] = 1
            else:
                corpus[(word_list[i-1], word)] += 1
        
    return corpus

In [84]:
truthful_bigram = get_bigram_counts(clean_text(truthful).split())
deceptive_bigram = get_bigram_counts(clean_text(deceptive).split())

In [9]:
def get_bigram_prob(corpus, bigram_to_test):
    total_words = 0
    for key in corpus:
        if key[0] == bigram_to_test[0]:
            total_words += corpus[key]

    return corpus[bigram_to_test]/total_words

In [76]:
get_bigram_prob(deceptive_bigram, ('I', 'am'))

0.01488933601609658

### Make Bigram Corpus

In [145]:
def get_smoothed_bigram_corpus(unigram_corpus, bigrams):
    df = pd.DataFrame(1, index = unigram_corpus, columns = unigram_corpus) 
    for bigram in bigrams:
        df.loc[bigram[0], bigram[1]] += bigrams[bigram]
    return df

In [146]:
df = pd.DataFrame(1, index =deceptive_unigram, columns =deceptive_unigram) 

In [147]:
get_smoothed_bigram_corpus(deceptive_unigram, deceptive_bigram)

,<s>,I,was,here,on,business,so,needed,to,get,...,recoup,rockin,ROYAL,gifts,allergy-friendly,tree,informal,tranquility,cleaners,beutiful
<s>,1,135,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
I,1,1,326,1,1,1,2,15,5,12,...,1,1,1,1,1,1,1,1,1,1
was,1,3,1,4,19,1,38,1,15,1,...,1,1,1,1,1,1,1,1,1,1
here,1,1,4,1,4,1,1,1,2,1,...,1,1,1,1,1,1,1,1,1,1
on,1,1,1,1,1,15,1,1,2,1,...,1,1,1,1,1,1,1,1,1,1
business,1,5,4,1,1,1,3,1,1,1,...,1,1,1,1,1,1,1,1,1,1
so,1,35,2,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
needed,1,1,2,1,1,1,1,1,11,1,...,1,1,1,1,1,1,1,1,1,1
to,1,1,3,1,2,1,3,1,1,75,...,1,1,1,1,1,1,1,1,1,1
get,1,1,1,1,3,1,1,1,15,1,...,1,1,1,1,1,1,1,1,1,1


In [34]:
def get_smoothed_bigram_prob(bigram, smoothed_bigram_corpus):
    return smoothed_bigram_corpus.loc[bigram[0], bigram[1]]/df.loc[bigram[0]].sum()

In [37]:
get_smoothed_bigram_prob(('I', 'am'), df)

0.0045114567256321972

## Part 4 – Validation

In [38]:
with open('./DATASET/validation/truthful.txt') as t, open('./DATASET/validation/deceptive.txt') as d:
    truthful_v = t.read()
    deceptive_v =d.read()

In [77]:
# replace all unknown words with <UNK> token
def check_for_unk_words(words, corpus):
    for i,word in enumerate(words):
        if word not in corpus:
              words[i] = '<UNK>'
    return words

## Validation Unigrams

In [143]:
# Cleaning & handling unknown words for truthful validation
cleaned_validation_truthful = check_for_unk_words(clean_text(truthful_v).split(), truthful_unigram)
truthful_unigram_v = Counter(cleaned_validation_truthful)

# Cleaning & handling unknown words for deceptive validation
cleaned_validation_deceptive = check_for_unk_words(clean_text(deceptive_v).split(), deceptive_unigram)
deceptive_unigram_v = Counter(cleaned_validation_deceptive)

## Validation Bigrams

In [136]:
truthful_bigram_v = get_bigram_counts(cleaned_validation_truthful)
deceptive_bigram_v = get_bigram_counts(cleaned_validation_deceptive)

In [153]:
get_smoothed_bigram_corpus(truthful_unigram_v, truthful_bigram_v)

,<s>,I,stayed,for,four,nights,while,attending,a,conference,...,NOTHING,training,opted,own,sterile,lives,indoor,movie,Mag,affordable
<s>,1,28,1,1,1,1,1,1,2,1,...,1,1,1,1,1,1,1,1,1,1
I,1,1,21,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
stayed,1,1,1,3,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
for,1,1,1,1,3,1,1,1,43,1,...,1,1,1,1,1,1,1,1,1,1
four,1,1,1,1,1,3,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
nights,1,1,1,4,1,1,3,1,1,1,...,1,1,1,1,1,1,1,1,1,1
while,1,3,1,1,1,1,1,3,2,1,...,1,1,1,1,1,1,1,1,1,1
attending,1,1,1,1,1,1,1,1,4,1,...,1,1,1,1,1,1,1,1,1,1
a,1,1,1,1,1,1,2,1,1,5,...,1,2,1,1,1,1,1,1,1,1
conference,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
